In [ ]:
import os
import cv2
import ast
import json
import subprocess
from glob import glob
from tqdm.notebook import tqdm
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Video

In [ ]:
import os
import cv2
import ast
import json
import subprocess
from glob import glob
from tqdm.notebook import tqdm
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Video

In [ ]:
# Root of input
INPUT_PATH = '../input/tensorflow-great-barrier-reef'
HEIGHT = 720 # image height
WIDTH  = 1280 # image width

In [ ]:
df_train = pd.read_csv(INPUT_PATH + '/train.csv')
display(df_train)
print(df_train.info())

In [ ]:
for video_id in df_train['video_id'].unique():
    print(f'video_id: {video_id}')
    print(f'w   annotations:  {sum(df_train[df_train["video_id"]==video_id]["annotations"] == "[]")}')
    print(f'w/o annotations:  {sum(df_train[df_train["video_id"]==video_id]["annotations"] != "[]")}\n')

In [ ]:
# Change the type of 'annotations' from str to list
df_train['annotations'] = df_train['annotations'].apply(ast.literal_eval) # str -> list
# Add columns of image path and number of bboxes
df_train['image_path'] = INPUT_PATH + '/train_images/video_' + df_train['video_id'].astype(str) + '/' + df_train['video_frame'].astype(str) + ".jpg"
df_train['num_bboxes'] = df_train['annotations'].apply(lambda x: len(x))
display(df_train)

In [ ]:
max_num_bboxes = max(df_train['num_bboxes'])
indexes = df_train[df_train['num_bboxes']==max_num_bboxes].index.values
print(f'Maximum number of bboxes in an image: {max_num_bboxes}')
display(df_train.iloc[indexes])

In [ ]:
indexes = [indexes[0], indexes[2]]

In [ ]:
def get_bboxes(annotations):
    """
    annotations: list of annotations
    return: bboxes as [x_min, y_min, x_max, y_max]
    """
    if len(annotations)==0:
        return []
    boxes = pd.DataFrame(annotations, columns=['x', 'y', 'width', 'height']).astype(np.int32).values
    # [x_min, y_min, w, h] -> [x_min, y_min, x_max, y_max]
    boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
    boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
    return boxes   

def plot_img_and_bbox(img_path, anntations):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(1, 1, figsize=(16,10))
    if len(annotations)>0:
        bboxes = get_bboxes(annotations)
        for i, box in enumerate(bboxes):
            # pur bbox on image
            cv2.rectangle(img,
                          (box[0], box[1]),
                          (box[2], box[3]),
                          color = (255, 0, 0),
                          thickness = 2)
            # numbering
            ax.text(box[0], box[1]-5, i+1, color='red')

    ax.set_axis_off()
    ax.imshow(img)


def zoom_bbox(img_path, annotations):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    bboxes = get_bboxes(annotations)
    
    col = 6
    row = np.ceil(len(bboxes)//6).astype(int)
    fig, ax = plt.subplots(row, col, figsize=(16,9))
    cnt = 0
    for i in range(row):
        if cnt >= len(bboxes):
            break
        for j in range(col):
            bbox = bboxes[cnt]
            sliced_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            ax[i,j].imshow(sliced_img)
            ax[i,j].set_title(cnt+1, color='red')
            ax[i,j].set_axis_off()
            cnt += 1
    plt.show() 

In [ ]:
samples = df_train.iloc[indexes].copy()
for idx, row in samples.iterrows():
    img_path    = row['image_path']
    annotations = row['annotations']
    print('image_id:', row['image_id'])
    # plot image with bboxes
    plot_img_and_bbox(img_path, annotations)
    # plot zoom of bboxes
    zoom_bbox(img_path, annotations)

In [ ]:
pd.read_csv(INPUT_PATH + '/train.csv').to_csv("submission.csv",index=False)

In [ ]:
df_train.to_csv("submission.csv",index=False)